# MLB.com stats crawler
https://github.com/whyhugo/Data-Analysis-in-Sports
> Please input the URL which can get from https://www.mlb.com/stats

In [1]:
import requests as rq
import pandas as pd
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

In [2]:
chrome_options = Options()
chrome_options.add_argument('--headless')

url = input('URL:')
driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')
html = driver.get(url)
sp = soup(driver.page_source, 'html.parser')

C:\Users\user\AppData\Local\Temp\ipykernel_18692\1222985024.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/programing/swiftx/chromedriver-win64/chromedriver.exe')


get player index
```python
index_list = []
index = sp.find_all(class_ = 'index-3cdMSKi7')
for i in range(0, len(index), 2):
    index_list.append(index[i].text)
```

In [3]:
# get player name
def get_player_name():
    player_list = []
    player = sp.find_all(class_ = 'full-3fV3c9pF')
    for i in range(0, len(player), 2):
        player_list.append(player[i].text + ' ' + player[i+1].text)
    return player_list

In [4]:
# get player's position
def get_player_position():
    position_list = []
    position = sp.find_all(class_ = 'position-28TbwVOg')
    for i in position:
        position_list.append(i.text)
    return position_list

In [5]:
def get_column_name():
    click_standard_button()
    sp = soup(driver.page_source, 'html.parser')
    
    data_column_name = ['PLAYER', 'POSITION', 'TEAM']
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
    temp = sp.find_all(class_ = 'bui-text cellheader selected-1vxxHvFg bui-text')
    for i in range(0, len(temp), 2):
        data_column_name.append(temp[i].text)

    click_expanded_button()
    sp = soup(driver.page_source, 'html.parser')
    temp = sp.find_all(class_ = 'bui-text cellheader bui-text')
    for i in range(4, len(temp), 2):
        data_column_name.append(temp[i].text)
        
    return data_column_name

In [6]:
# click standard button
def click_standard_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[1]/button').click()
    time.sleep(3)
    sp = soup(driver.page_source, 'html.parser')

In [7]:
# click expanded button
def click_expanded_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[1]/div[2]/div/div[1]/div/div[2]/button').click()
    time.sleep(3)

In [8]:
# click next page button
def click_next_page_button():
    time.sleep(3)
    driver.find_element('xpath', '//*[@id="stats-app-root"]/section/section/div[3]/div[2]/div/div/div[2]/button').click()
    time.sleep(3)
    sp = soup(driver.page_source, 'html.parser')

In [9]:
# get standard data
def get_data():
    data = []
    
    click_standard_button()
    sp = soup(driver.page_source, 'html.parser')
    standard_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        standard_data.append(i.text)
    
    click_expanded_button()
    sp = soup(driver.page_source, 'html.parser')
    expanded_data = []
    tt = sp.select('#stats-app-root tr td')
    for i in tt:
        expanded_data.append(i.text)

    for i in range(len(standard_data)//17):
        data.extend(standard_data[i*17:i*17+17])
        data.extend(expanded_data[i*16+1:i*16+16])

    return data

```python
html = driver.get(url) 
# 有時候會 get 不完整，可以用這邊 reload 一次 XD
```

In [10]:
try:
    df_list = []
    while True:
        sp = soup(driver.page_source, 'html.parser')
        print('page start')
        time.sleep(2)
        name = get_player_name()
        time.sleep(2)
        position = get_player_position()
        time.sleep(2)
        current_data = get_data()
        time.sleep(2)
        for i in range(len(name)):
            row = []
            row.append(name[i])
            row.append(position[i])
            row.extend(current_data[i*32:i*32+32])
            df_list.append(row)
        click_next_page_button()
except:
    print('This is the last page :)')

page start
page start
This is the last page :)


In [11]:
df = pd.DataFrame(df_list, columns=get_column_name())
df

,PLAYER,POSITION,TEAM,G,AB,R,H,2B,3B,HR,...,GO/AO,XBH,TB,IBB,BABIP,ISO,AB/HR,BB/K,BB%,SO%
0,Giancarlo Stanton,RF,MIA,116,425,62,106,26,0,24,...,1.12,50,204,5,.313,.231,17.71,.529,.147,.278
1,Brian Flynn,P,MIA,4,5,0,2,0,0,0,...,1.00,0,2,0,.667,.000,-.--,.---,.000,.333
2,Henderson Alvarez,P,MIA,17,30,2,9,3,0,1,...,2.50,4,15,0,.444,.200,30.00,.---,.000,.343
3,Christian Yelich,LF,MIA,62,240,34,69,12,1,4,...,3.24,17,95,1,.380,.108,60.00,.470,.114,.242
4,Logan Morrison,1B,MIA,85,293,32,71,13,4,6,...,1.08,23,110,5,.281,.133,48.83,.679,.114,.168
5,Justin Ruggiano,CF,MIA,128,424,49,94,18,1,18,...,1.17,37,168,1,.260,.175,23.56,.360,.087,.242
6,Marcell Ozuna,RF,MIA,70,275,31,73,17,4,3,...,1.15,24,107,0,.326,.124,91.67,.228,.045,.196
7,Joe Mahoney,1B,MIA,9,29,2,8,1,0,1,...,0.70,2,12,0,.292,.138,29.00,.---,.000,.138
8,Derek Dietrich,2B,MIA,57,215,32,46,10,2,9,...,0.92,21,87,1,.247,.191,23.89,.196,.047,.240
9,Chris Coghlan,X,MIA,70,195,10,50,10,3,1,...,1.29,14,69,1,.322,.097,195.00,.395,.079,.201


In [12]:
df.to_csv(f'{url[26:-20]}_{url[-19:-15]}.csv', index=False, encoding='utf-8')

In [13]:
driver.quit()